In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
from matplotlib import pyplot as plt

In [2]:
torch.cuda.empty_cache()

In [3]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]
)

dataset = ImageFolder("train/", transform= transform)
len(dataset)

88011

In [5]:
seed = 892
torch.manual_seed(seed)

In [6]:
batch_size = 500
train_size = 74000

val_size = len(dataset) - train_size


val_dataset, train_dataset = data.random_split(dataset, lengths=[val_size, train_size])

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=1)
val_loader = data.DataLoader(val_dataset, batch_size=100, pin_memory=True, num_workers=1)
print(len(val_dataset), len(train_dataset))
int(len(train_dataset)/batch_size)

14011 74000


148

In [7]:
class ImageClassifierNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.loss = nn.CrossEntropyLoss()

    def train_step(self, input, labels):
        input = input.to(device=device)#, non_blocking=True)
        labels = labels.to(device=device)#, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(preds, labels)

        return loss

    def val_step(self, input, labels):

        input = input.to(device=device)#, non_blocking=True)
        labels = labels.to(device=device)#, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(preds, labels)
        accuracy = self._accuracy(preds, labels)

        return {"loss":loss.detach(), "accuracy":accuracy}


    def val_epoch_end(self, preformance_measurement_data):

        accuracy = [x["accuracy"].cpu().numpy() for x in preformance_measurement_data]
        avg_accuracy = np.mean(accuracy)

        loss = [x["loss"].cpu().numpy() for x in preformance_measurement_data]
        avg_loss = np.mean(loss)

        return avg_loss, avg_accuracy


    def _accuracy(self, preds, labels):
        batch_size = len(preds)

        pred_indices = torch.argmax(preds, dim=1)
        return torch.tensor(torch.sum(pred_indices == labels).item() / batch_size)





In [8]:
class ResNetBlock(nn.Module):
    """ RestNet block with BN and full pre-activation """
    def __init__(self, in_channels, out_channels_conv1, out_channels_conv2, kernel_size=[3, 3], stride=[1,1], padding=[1,1]):
        super().__init__()

        self.network = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels_conv1, kernel_size=kernel_size[0], stride=stride[0], padding=padding[0]),
            nn.ReLU(),
            nn.Conv2d(out_channels_conv1, out_channels_conv2, kernel_size=kernel_size[1], stride=stride[1], padding=padding[1])
        )


    def forward(self, input):
        return self.network(input) + input




class ResNetBlockChangeDepth(ResNetBlock):
    """ RestNet block with depth resize """
    def __init__(self, in_channels, out_channels_conv1, out_channels_conv2, res_conv_kernel_size=1, res_conv_stride=1, res_conv_padding=0, kernel_size=[3, 3], stride=[1,1], padding=[1,1]):

        super().__init__(in_channels, out_channels_conv1, out_channels_conv2, kernel_size, stride, padding)

        self.conv = nn.Conv2d(in_channels, out_channels_conv2, kernel_size=res_conv_kernel_size, stride=res_conv_stride, padding=res_conv_padding)

    def forward(self, input):
        return self.network(input) + self.conv(input)



class Wrapper(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


In [9]:
class ResNet(ImageClassifierNetwork):

    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.resnet_block1_1 = ResNetBlock(64, 64, 64)
        self.resnet_block1_2 = ResNetBlockChangeDepth(64, 128, 128)
        self.resnet_block2_1 = ResNetBlock(128, 128, 128)
        self.resnet_block2_2 = ResNetBlockChangeDepth(128, 256, 256)


        self.network = nn.Sequential(

            # Prep 3x64x64
            nn.Conv2d(in_channels, 32, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 32x32x32

            nn.Conv2d(32, 64, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 64x16x16

            Wrapper(self.resnet_block1_1),
            Wrapper(self.resnet_block1_2),
            nn.MaxPool2d(2, 2), # 128x8x8

            Wrapper(self.resnet_block2_1),
            Wrapper(self.resnet_block2_2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AvgPool2d(2, 2), # 256x4x4

            nn.Flatten(),
            # Fully connected layer
            nn.Linear(4*1024, num_classes),
            # nn.Dropout(0.5),
            # nn.ReLU(),
            # nn.Linear(512, num_classes)

        )


    def forward(self, input):
        return self.network(input)


        # test_model
        # self.network = nn.Sequential(

        #     # Prep 3x64x64
        #     nn.Conv2d(in_channels, 64, kernel_size=5, padding=2),
        #     nn.BatchNorm2d(64),
        #     nn.ReLU(),
        #     nn.MaxPool2d(2, 2),  # 64x32x32

        #     nn.Conv2d(64, 128, kernel_size=5, padding=2),
        #     nn.BatchNorm2d(64),
        #     nn.ReLU(),


        #     Wrapper(self.resnet_block1),
        #     nn.MaxPool2d(2, 2), # 128x8x8

        #     Wrapper(self.resnet_block2),
        #     Wrapper(self.resnet_block3),

        #     nn.ReLU(),
        #     nn.MaxPool2d(2, 2), # 256x4x4

        #     nn.Flatten(),
        #     # Fully connected layer
        #     nn.Linear(4096, num_classes),
        #     # nn.Dropout(0.5),
        #     # nn.ReLU(),
        #     # nn.Linear(2048, num_classes)

        # )

In [10]:
class ResNet_2(ImageClassifierNetwork):

    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.resnet_block1_1 = ResNetBlock(64, 64, 64)
        self.resnet_block1_2 = ResNetBlockChangeDepth(64, 128, 128)
        self.resnet_block2_1 = ResNetBlock(128, 128, 128)
        self.resnet_block2_2 = ResNetBlockChangeDepth(256, 256, 256)
        self.resnet_block3_1 = ResNetBlock(256, 256, 256)
        self.resnet_block3_2 = ResNetBlockChangeDepth(256, 512, 512)



        self.network = nn.Sequential(

            # Prep 3x64x64
            nn.Conv2d(in_channels, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 32x32x32

            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            Wrapper(self.resnet_block1_1),
            Wrapper(self.resnet_block1_2),
            nn.MaxPool2d(2, 2),  # 128x16x16

            Wrapper(self.resnet_block2_1),
            Wrapper(self.resnet_block2_2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 256x8x8

            Wrapper(self.resnet_block3_1),
            Wrapper(self.resnet_block3_2),
            nn.ReLU(),
            nn.AvgPool2d(2, 2), # 512x4x4


            nn.Flatten(),
            # Fully connected layer
            nn.Linear(8192, 2048),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(2048, num_classes)

        )


    def forward(self, input):
        return self.network(input)




In [11]:
model = ResNet_2(3, 50)
# model.load_state_dict(torch.load("test_model.pt"))

model.to(device)

AttributeError: 'ResNet_2' object has no attribute 'resnet_block1'

In [ ]:
# RestNet +- 7epoch;; RestNet_2 +- 12epoch
epoch = 7

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
lr_scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 1e-2, epochs=epoch, steps_per_epoch=int(len(train_dataset)/batch_size))

In [ ]:
params_sum = 0
for params in model.parameters():
    params_sum+=params.view(-1).size(0)
params_sum

1776978

In [ ]:
def eval(val_loader):
    history_val = []
    for input, label in val_loader:
        history += [model.val_step(input, label)]

    loss_val, acc_val = model.val_epoch_end(history_val)

    return loss_val, acc_val

In [ ]:
def train(train_dataloader, epoch):
    model.train(mode=True)

    #TODO Dadac agumentację dancyh

    epoch_loss_T = []
    batch_loss_T = []
    epoch_loss_V = []
    epoch_acc_V = []


    for ep in range(epoch):

        # i=1
        for input, label in train_dataloader:
            # print(i)
            # i+=1
            loss = model.train_step(input, label)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            lr_scheduler.step()

            batch_loss_T += [loss.item()]

        epoch_loss_T += [np.mean(batch_loss_T)]
        batch_loss_T = []

        loss_val, acc_val = eval(val_loader)
        epoch_loss_V += [loss_val]
        epoch_acc_V += [acc_val]

        print(f'Epoch {ep}; TLoss: {epoch_loss_T[ep]}; Vloss: {loss_val}; Acc: {acc_val}')


    model.eval()
    return epoch_loss_T, epoch_loss_V, epoch_acc_V

In [ ]:
loss_T, loss_V, acc_V = train(train_loader, epoch)


Epoch 0; TLoss: 2.7286988174593128; Vloss: 2.3550772666931152; Acc: 0.36447450518608093
Epoch 1; TLoss: 2.157891220337636; Vloss: 2.2097761631011963; Acc: 0.41582202911376953
Epoch 2; TLoss: 1.9387757222394686; Vloss: 1.975256085395813; Acc: 0.46824631094932556
Epoch 3; TLoss: 1.6984277117896724; Vloss: 1.8465632200241089; Acc: 0.49909085035324097
Epoch 4; TLoss: 1.5084868282885164; Vloss: 1.734597086906433; Acc: 0.5293939709663391
Epoch 5; TLoss: 1.3252588462185215; Vloss: 1.6647933721542358; Acc: 0.5437846779823303
Epoch 6; TLoss: 1.1549671594355557; Vloss: 1.6353799104690552; Acc: 0.561025083065033
Epoch 7; TLoss: 0.9931440244655352; Vloss: 1.6594702005386353; Acc: 0.562301754951477
Epoch 8; TLoss: 0.8599538948084857; Vloss: 1.697561502456665; Acc: 0.5651257038116455
Epoch 9; TLoss: 0.7305966067958523; Vloss: 1.7883249521255493; Acc: 0.5561186075210571
Epoch 10; TLoss: 0.5699805961670102; Vloss: 1.863176941871643; Acc: 0.5556285381317139
Epoch 11; TLoss: 0.4158387524453369; Vloss: 1

KeyboardInterrupt: 

In [ ]:
def plot_results(loss_T, loss_V, acc_V):

    # fig = plt.figure
    plt.plot(loss_T, label="Train")
    plt.plot(loss_V, label="Val")
    plt.title("Loss")
    plt.legend()
    plt.show()

    # fig2 = plt.figure
    plt.plot(acc_V, label="Accuracy")
    plt.legend()
    plt.show()

plot_results(loss_T, loss_V, acc_V)

NameError: name 'loss_T' is not defined

In [ ]:
# model = ResNet(3, 50)
# model.to(device)
# model.load_state_dict(torch.load("test_model.pt"))

In [ ]:
def eval(val_loader):
    history = []
    for input, label in val_loader:
        history += [model.val_step(input, label)]

    result = model.val_epoch_end(history)

    return result, history

In [ ]:
result, history = eval(val_loader)
result

{'loss': {1.6540478}, 'accuracy': 0.6124435}

In [ ]:
# torch.save(model.state_dict(), "10epoch_standard_model1_61acc.pt")


In [ ]:
# model = ResNet(3, 50)
# model.load_state_dict(torch.load("test_model.pt"))
